# ConversationEntityMemory

엔티티 메모리는 대화에서 특정 엔티티에 대한 주어진 사실을 기억합니다.

엔티티 메모리는 엔티티에 대한 정보를 추출하고(LLM 사용) 시간이 지남에 따라 해당 엔티티에 대한 지식을 축적합니다(역시 LLM 사용).


In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH05-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Memory


In [4]:
# Bedrock 설정 및 Util 함수 설정
from langchain_aws import ChatBedrockConverse
import os
from Util.stream_utils import print_stream_content, get_stream_content

# 객체 생성
llm = ChatBedrockConverse(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    model="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
)

In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

Entity 메모리를 효과적으로 사용하기 위하여, 제공되는 프롬프트를 사용합니다.


In [6]:
# Entity Memory를 사용하는 프롬프트 내용을 출력합니다.
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

In [7]:
memory = ConversationEntityMemory(llm=llm)
memory.save_context(
    {"input": "테디와 셜리는 한 회사에서 일하는 동료입니다. 테디는 개발자이고 셜리는 디자이너입니다. 그들은 최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다."},
    {"output": """테디와 셜리의 창업 계획이 흥미롭네요! 개발자와 디자이너의 조합은 스타트업에서 매우 강력한 파트너십이 될 수 있습니다.

이들이 성공적인 창업을 위해 고려해볼 만한 몇 가지 사항들이 있을 것 같은데요:

**강점 활용:**
- 테디의 기술적 역량과 셜리의 디자인 감각을 결합한 제품/서비스 개발
- 서로 다른 전문 분야로 인한 상호 보완적 관계

**준비 사항:**
- 사업 아이템 구체화 및 시장 조사
- 초기 자금 확보 방안
- 역할 분담 및 지분 구조 설정
- 법적 절차 (사업자 등록, 회사 설립 등)

테디와 셜리가 어떤 분야의 사업을 계획하고 있는지, 또는 창업 과정에서 어떤 부분에 대해 더 자세히 알고 싶으신가요?"""},
)

C:\Users\younghl\AppData\Local\Temp\ipykernel_7648\799592553.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationEntityMemory(llm=llm)
c:\Users\younghl\CursorProjects\langchain-kr\.venv\Lib\site-packages\pydantic\main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [8]:
memory.load_memory_variables({"input": "테디는 무얼하려고 합니까?"})

{'history': 'Human: 테디와 셜리는 한 회사에서 일하는 동료입니다. 테디는 개발자이고 셜리는 디자이너입니다. 그들은 최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다.\nAI: 테디와 셜리의 창업 계획이 흥미롭네요! 개발자와 디자이너의 조합은 스타트업에서 매우 강력한 파트너십이 될 수 있습니다.\n\n이들이 성공적인 창업을 위해 고려해볼 만한 몇 가지 사항들이 있을 것 같은데요:\n\n**강점 활용:**\n- 테디의 기술적 역량과 셜리의 디자인 감각을 결합한 제품/서비스 개발\n- 서로 다른 전문 분야로 인한 상호 보완적 관계\n\n**준비 사항:**\n- 사업 아이템 구체화 및 시장 조사\n- 초기 자금 확보 방안\n- 역할 분담 및 지분 구조 설정\n- 법적 절차 (사업자 등록, 회사 설립 등)\n\n테디와 셜리가 어떤 분야의 사업을 계획하고 있는지, 또는 창업 과정에서 어떤 부분에 대해 더 자세히 알고 싶으신가요?',
 'entities': {'테디': ''}}

In [9]:
# ConversationChain 을 생성합니다.
conversation = ConversationChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm),
)

C:\Users\younghl\AppData\Local\Temp\ipykernel_7648\2679377046.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


대화를 시작합니다.

입력한 대화를 바탕으로 `ConversationEntityMemory` 는 주요 Entity 정보를 별도로 저장합니다.


In [10]:
conversation.predict(
    input="테디와 셜리는 한 회사에서 일하는 동료입니다."
    "테디는 개발자이고 셜리는 디자이너입니다. "
    "그들은 최근 회사에서 일하는 것을 그만두고 자신들의 회사를 차릴 계획을 세우고 있습니다."
)

'테디와 셜리가 창업을 준비하고 있다니 흥미로운 상황이네요! 개발자와 디자이너의 조합은 스타트업에서 매우 강력한 팀워크를 만들어낼 수 있습니다.\n\n그들의 창업 계획에 대해 더 자세히 알고 싶습니다:\n\n- 어떤 분야나 서비스로 창업을 계획하고 있나요?\n- 현재 어느 단계까지 준비가 진행되었나요? (아이디어 구상, 사업계획서 작성, 자금 조달 등)\n- 두 사람이 각각 어떤 역할을 맡게 될 예정인가요?\n\n테디의 개발 역량과 셜리의 디자인 역량이 잘 결합된다면, 사용자 경험이 뛰어난 제품이나 서비스를 만들어낼 수 있을 것 같습니다. 창업 과정에서 도움이 필요한 부분이 있다면 언제든 말씀해 주세요!'

Entity는 `memory.entity_store.store` 에서 확인할 수 있습니다.


In [11]:
# entity memory 를 출력합니다.
conversation.memory.entity_store.store

{'테디': '테디는 개발자로 일하고 있으며, 동료인 디자이너 셜리와 함께 현재 회사를 그만두고 창업할 계획을 세우고 있습니다.',
 '셜리': '셜리는 디자이너로 일하고 있으며, 동료인 테디와 함께 회사를 그만두고 창업을 계획하고 있습니다.'}